![alt text](https://cienciadosdados.com/images/2022/HEADER.png)
# <font color='Blue'>Dicas e Truques de Ciência de Dados</font>
# <font color='green'>Interpretabilidade do Modelo com o pacote Eli5 </font>

### Melhores Variáveis para sua Máquina Preditiva


![](https://i.gifer.com/1kLq.gif)


# Competições de Ciência de Dados

![](https://thumbs.gfycat.com/GiddyWhoppingCockroach-size_restricted.gif)

**Link**: 

https://machinehack.com/hackathon/data_hack_mathcothon_car_price_prediction_challenge/overview

### Importanto bibliotecas/pacotes

In [1]:
import pandas as pd 
import numpy as np

### Conexão com os Dados

In [3]:
df = pd.read_csv('car_data.csv')

### Explorando os Dados

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
col = df.columns

In [ ]:
print(df.Seller_Type.unique())
print(df.Transmission.unique())
print(df.Owner.unique())

In [ ]:
## check missing values or null values
df.isna().sum()

In [ ]:
df.describe()

In [ ]:
df.corr()

### Renomeando as Variáveis

In [10]:
final_df = df[['Year', 'Selling_Price', 'Present_Price', 'Kms_Driven', 'Fuel_Type', 'Seller_Type', 'Transmission', 'Owner']]

In [ ]:
final_df.head()

In [ ]:
final_df.head()

### Engenharia de Atributos - Criando a variável Idade do carro

In [12]:
final_df['Current_Year'] = 2021

In [14]:
final_df['no._year'] = final_df['Current_Year'] - final_df['Year']

In [ ]:
final_df.head()

In [16]:
final_df.drop(['Year'], axis=1, inplace=True)

In [ ]:
final_df.head()

In [18]:
final_df.drop(['Current_Year'], axis=1, inplace=True)

In [ ]:
final_df.head()

### Fazendo OneHotEncoding: Criando variáveis numéricas

In [20]:
final_df = pd.get_dummies(final_df, drop_first=True)

In [ ]:
final_df.head()

### Verificando variáveis e suas correlações

In [22]:
import seaborn as sns

In [ ]:
sns.pairplot(final_df)

In [24]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
corrmat = final_df.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(20, 20))
g = sns.heatmap(final_df[top_corr_features].corr(), annot=True, cmap="RdYlGn")

### Separando (split) das Variáveis: Separando o Target(preço de venda) das variáveis explicativas (X)

In [26]:
# independent features and dependent features
X = final_df.iloc[:, 1:]
y = final_df.iloc[:, 0]

In [ ]:
X.head()

In [ ]:
y.head()

### Quais variáveis são as mais importantes?

![](https://endomarketing.tv/wp-content/uploads/2016/04/endomarketing-progic-importancia-da-tv-corporativa-712x350.gif)

In [ ]:
## feature important
from sklearn.ensemble import ExtraTreesRegressor
model= ExtraTreesRegressor()
model.fit(X, y)

In [ ]:
model.get_params()

In [ ]:
model.feature_importances_

In [ ]:
feature_importances = pd.Series(model.feature_importances_, index=X.columns)
# feature_importances.nlargest(5).plot(kind='barh')
feature_importances.plot(kind='barh')
plt.show()

In [62]:
from sklearn.model_selection import train_test_split

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [36]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

## Tunnando Hyperparametros com Random Search CV

In [37]:
# Randomized Search CV

## Number of trees in ramdom forest
n_estimators = [int(x) for x in np.linspace(start=100, stop=1200, num=12)]
## Number of features to consider at every split
max_features = ['auto', 'sqrt']
## Maximum number of level in tree
max_depth = [int(x) for x in np.linspace(5, 30, num=6)]
## Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
## Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]

In [38]:
## it helps us to find out the best parameters
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
## create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}
print(random_grid)


In [40]:
rf_random = RandomizedSearchCV(estimator=rf, 
                               param_distributions=random_grid, 
                               scoring='neg_mean_squared_error',
                               n_iter=10, 
                               cv=5, 
                               verbose=2, 
                               random_state=42, 
                               n_jobs=1)

In [ ]:
rf_random.fit(X_train, y_train)

### Avaliando os Melhores Parâmetros

In [ ]:
print('Best Params:', rf_random.best_params_)

### Avaliando a Máquina Preditiva

In [ ]:
print('Best CV Score:', -rf_random.best_score_)

Link:

https://eli5.readthedocs.io/en/latest/

https://focanocliente.com.br/eli5-explain-like-im-5/

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQwGLg2s22eueRcjS3I3-vQg2LQrJTohNZ2HGBSg1AmOMM_4p9H6CyhGKLYUooBxaHlxNI&usqp=CAU)

### Interpretabilidade da Máquina Preditiva

In [ ]:
# Instalação
!pip install eli5

In [49]:
import eli5
from eli5.sklearn import PermutationImportance

In [ ]:
perm = PermutationImportance(rf_random, random_state=1).fit(X_test,  y_test)
eli5.show_weights(perm, feature_names = X_test.columns.tolist())

**Inferência:**

  > Pode-se ver acima que o preço de um carro depende principalmente do "Present_Price" seguida da "Idade", "Quilometragem" e "Tipo de Vendedor".



In [42]:
pred = rf_random.predict(X_test)

In [ ]:
sns.distplot(y_test-pred)

In [ ]:
import pickle
file = open('random_forest_regression_model.pkl', 'wb')
pickle.dump(rf_random, file)

#Fim

## Valeu!

### # Onde me Encontrar - Ciência dos Dados <a href="https://linktr.ee/cienciadosdados">https://linktr.ee/cienciadosdados</a>



In [64]:
from IPython.core.display import HTML
HTML('<iframe width="380" height="220" src="https://www.youtube.com/embed/Nli7I7JAVRE" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')